# Neural network Tuning

In [2]:
pip install tensorflow==2.6.0


     -------------------------------------- 423.3/423.3 MB 1.3 MB/s eta 0:00:00
     ---------------------------------------- 2.7/2.7 MB 1.5 MB/s eta 0:00:00
     ---------------------------------------- 13.3/13.3 MB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 42.6/42.6 kB ? eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     -------------------------------------- 132.0/132.0 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for clang: filename=clang-5.0-py3-none-any.whl size=30681 sha256=69e49d6f3dc2674d7d768cfec3938fc297d37737a9816aa0f82e8316d8364db5
  Stored in directory: c:\users\hp\appdata\local\pip\cache\wheels\3a\ce\7a\27094f689461801c934296d07078773603663dfcaca63bb064
  Created wheel for termcolor: 

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\HP\\anaconda3\\Lib\\site-packages\\~rapt\\_wrappers.cp39-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



In [3]:
# Importing necesary packages
import numpy
import pandas as pd
from sklearn.model_selection import GridSearchCV 
from keras.models import Sequential
from keras.layers import Dense
import warnings
warnings.filterwarnings('ignore')
from keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

ModuleNotFoundError: No module named 'keras.wrappers'

# part 1 - Tuning batch and epoch

In [ ]:
# load the dataset for India diabetes
data = pd.read_csv('diabetes.csv')

In [ ]:
#viewing data
data.head()

In [ ]:
# split into predictors and response
X = data.iloc[:,0:8] #predictors
Y = data.iloc[:,8] #response

In [ ]:
# Function to create model,for KerasClassifier
def create_my_model():
    #defining my model
    mymodel = Sequential()
    mymodel.add(Dense(12, input_dim=8, activation='relu'))
    mymodel.add(Dense(1, activation='sigmoid'))
    
    # Compile the model
    mymodel.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return mymodel

In [ ]:
# create model
model = KerasClassifier(build_fn=create_my_model)

In [ ]:
# define the grid search parameters
batchSize = [10, 20, 40, 60, 80, 100]
epochs = [10, 30, 50]

In [ ]:
parameter_grid = dict(batch_size=batchSize, epochs=epochs)


mygrid = GridSearchCV(estimator=model, param_grid=parameter_grid, n_jobs=-1, cv=3)
grid_result = mygrid.fit(X, Y)

In [ ]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

# part 2- Optimizing Activation Function

In [ ]:
# Function to create model
def create_my_model(optimizer='adam'):
    # create model
    mymodel = Sequential()
    mymodel.add(Dense(12, input_dim=8, activation='relu'))
    mymodel.add(Dense(1, activation='sigmoid'))
    # Compile model
    mymodel.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return mymodel

In [ ]:
# create model
model = KerasClassifier(build_fn=create_my_model, epochs=50, batch_size=10)

In [ ]:
# define the grid search parameters
optimizer = ['SGD','Adadelta', 'RMSprop', 'Adagrad','Adam']
parameter_grid = dict(optimizer=optimizer)

In [ ]:
grid = GridSearchCV(estimator=model, param_grid=parameter_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, Y)

In [ ]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

# How can you improve accuracy of the model? 

- changing the number of layers and Hyperparameters
- increase number of epocks to 100 rather than 50


``` def create_my_model(learning_rate=0.001, batch_size=32):
    mymodel = Sequential()
    mymodel.add(Dense(64, input_dim=8, activation='relu'))
    mymodel.add(Dense(32, activation='relu'))
    mymodel.add(Dense(16, activation='relu'))
    mymodel.add(Dense(1, activation='sigmoid'))
    
    
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    
    
    mymodel.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return mymodel
```